# MUSO Groups Close_OUT Reports

---

Per groups:

1. date graduation is valid and in the reporting.
2. number of member.
3. number of groups
5. number of indirect beneficiaires.

## 

## Preliminaries

In [1]:
from pandas import read_sql_query, read_excel
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from decouple import config
from sqlalchemy import create_engine, text
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.core.interactiveshell import InteractiveShell





InteractiveShell.ast_node_interactivity = 'all'

load_dotenv()
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')



In [2]:
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

In [3]:
# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

In [4]:
mhc22 = read_sql_query(text(MHC22),engine.connect(),parse_dates=True)
nomhc22 = read_sql_query(text(NOMHC22),engine.connect(),parse_dates=True)
mh22 = read_sql_query(text(MUSO_HOUSEHOLD_22),engine.connect(),parse_dates=True)

In [5]:
engine.dispose()

In [6]:
#load_dotenv('./id_cc.env')
email = config('COMCARE_EMAIL')
password_cc = config('COMCARE_PASSWORD')

driver = webdriver.Chrome()
driver.implicitly_wait(1000)

#Creating login function
def muso_beneficiaries():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce97cc9d15/'
    )
    #driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    #driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#Muso beneficiaries
muso_beneficiaries()

#Download the database "All gardens"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()    
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()    

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def muso_groups():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/91fe1e2fc809dfd56a29bfce973b4c0d/'
    )
    
muso_groups()

#Download the database "Household count"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

In [7]:
muso_benef = read_excel(f"~/Downloads/muso_beneficiaries {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", sheet_name = 'Cases')
muso_group = read_excel(f"~/Downloads/muso_groupes {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")

## Groupes a desactiver


In [8]:
groupadesactiver = read_excel("./groupe_adesactiver_muso/groupes_adesactiver_finale.xlsx", sheet_name=1)

In [9]:
groupadesactiver.columns

Index(['Date de graduation (YYYY-MM-DD)', 'Departement', 'Commune',
       'Nom du groupe', 'numero_groupe', 'OFFICE', 'reason_currated',
       'Cause/Raison', 'caseid_grpes', 'effectively_closed', 'has_dated',
       'open_weird'],
      dtype='object')

In [10]:
groupadesactiver.rename(
    columns={"caseid_grpes":"caseid_group"},
    inplace=True
)

In [11]:
gad = groupadesactiver[['caseid_group','OFFICE','has_dated']]

In [12]:
gad

,caseid_group,OFFICE,has_dated
0,fff1657eb0334ae982af607bf9c0df13,GON,no
1,fe8147bcf49d4db08032116600e3a6e8,CAY,yes
2,fe2b4775-8e9a-488f-9bef-14d1a09fc122,JER,yes
3,fd36188df2a84b96bb092cafb8f8c8d0,CAY,yes
4,fc2cfd35-e89e-4366-94a2-4ffb1db22fbe,PAP,no
...,...,...,...
126,032c2252-06d9-44c0-b2e0-45821b67333f,CAY,yes
127,02bb70e51c9241218ca77c548874f52e,CAP,no
128,014181e8df164f8e89ab0a5f3125d606,CAY,no
129,012d6d9174da449abc1d722864954394,GON,no


## groups actifs et leurs membre

In [13]:
muso_benef.rename(columns = {'indices.muso_groupes': 'caseid_group'}, inplace = True)

In [14]:
ben_muso = muso_benef[['closed','caseid', 'caseid_group','why_remove_the_member']]

In [15]:
muso_group.rename(columns = {'is_graduated': 'graduated_group',
                            'is_inactive': 'inactive_group',
                            'closed': 'closed_group',
                            'caseid': 'caseid_group',
                            'office': 'office_group'}, inplace = True)

In [16]:
group_info = muso_group[['caseid_group', 'office_name', 'office_group', 'graduated_group', 'inactive_group', 'closed_group']]

In [17]:
group_info.shape

(2272, 6)

In [18]:
ben_muso.shape

(54080, 4)

In [19]:
ben_with_group_info = ben_muso.merge(group_info,on="caseid_group",how="left")

In [20]:
ben_with_group_info.shape

(54080, 9)

### groupe actif et les benficiaires

In [21]:
active_goup_and_ben = ben_with_group_info[
    (ben_with_group_info['why_remove_the_member'] == '---') & 
    (ben_with_group_info['graduated_group'] != '1') &
    (ben_with_group_info['inactive_group'] != '1') & 
    (ben_with_group_info['closed_group'] != True)
]
active_goup_and_ben.shape

(25155, 9)

In [22]:
active_musogroup = muso_group[
    (muso_group['graduated_group'] != '1') &
    (muso_group['inactive_group'] != '1') & 
    (muso_group['closed_group'] != True)
]
active_musogroup.shape

(1061, 57)

# how many groups

In [23]:
gad.shape
gad.columns

(131, 3)

Index(['caseid_group', 'OFFICE', 'has_dated'], dtype='object')

In [24]:
muso_group[muso_group.caseid_group.isin(gad.caseid_group)].shape
#active_musogroup[active_musogroup.caseid_group.isin(gad.caseid_group)].shape

(131, 57)

### how many beneficiaires

In [25]:
ben_with_group_info[ben_with_group_info.caseid_group.isin(gad.caseid_group)].shape
ben_muso[ben_muso.caseid_group.isin(gad.caseid_group)].shape
muso_benef[muso_benef.caseid_group.isin(gad.caseid_group)].shape


(4155, 9)

(4155, 4)

(4155, 76)

### How many indirect

In [26]:
future_lost_ben = ben_muso[ben_muso.caseid_group.isin(gad.caseid_group)]
flb = muso_benef[muso_benef.caseid_group.isin(gad.caseid_group)]

In [27]:
mh22[mh22.caseid.isin(flb.caseid)].shape
mh22[mh22.caseid.isin(future_lost_ben.caseid)].shape

(6360, 15)

(6360, 15)

In [28]:
from pandas import DataFrame

In [29]:
DataFrame.from_dict(
    {
        "#groupes": [gad.shape[0]],
        "#members": [ben_muso[ben_muso.caseid_group.isin(gad.caseid_group)].shape[0]],
        "#HC22": [mh22[mh22.caseid.isin(future_lost_ben.caseid)].shape[0]],
    },
    orient="index",
    columns=["Total"]
)

,Total
#groupes,131
#members,4155
#HC22,6360


In [30]:
DataFrame.from_dict(
    {
        "#groupes": [gad.shape[0]],
        "#members": [ben_muso[ben_muso.caseid_group.isin(gad.caseid_group)].shape[0]],
        "#HC22": [mh22[mh22.caseid.isin(future_lost_ben.caseid)].shape[0]],
    },
    orient="index",
    columns=["Total"]
).to_excel('./groupe_adesactiver_muso/resume_gad.xlsx',index=True)

In [31]:
driver.quit()
